# Singleton testing

> Dealing with the singleton's problems when unit testing.

The singleton pattern is considered a "bad" design pattern and even the [Gang of Four](https://en.wikipedia.org/wiki/Design_Patterns) considered dropping it from [Design Patterns](https://en.wikipedia.org/wiki/Design_Patterns) because it's usage is almost always a "design smell".

We will now take a look at the singleton's problems and how they can be solved.

For this example we will build a database of capitals and their populations with the metaclass implementation of the singleton. We will use an external txt file to load up the database.

In [1]:
class Singleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Database(metaclass=Singleton):
    def __init__(self):
        self.population = {}
        f = open('capitals.txt', 'r')
        lines = f.readlines()
        for i in range(0, len(lines), 2):
            self.population[lines[i].strip()] = int(lines[i + 1].strip())
        f.close()

Now let's create a higher level module which uses the database for something. We will create a record finder that will calculate the total population of several different cities provided by the end user.

In [2]:
class SingletonRecordFinder:
    def total_population(self, cities):
        result = 0
        for c in cities:
            result += Database().population[c]
        return result

Now let's suppose that we want to test `SingletonRecordFinder`. We will write some unit tests which use it.

In [4]:
import unittest

class SingletonTests(unittest.TestCase):
    def test_is_singleton(self):
        db = Database()
        db2 = Database()
        self.assertEqual(db, db2)
        
    def test_singleton_total_population(self):
        """ This tests on a live database!! """
        rf = SingletonRecordFinder()
        names = ['Seoul', 'Mexico City']
        tp = rf.total_population(names)
        self.assertEqual(tp, 17500000 + 17400000) # These values could change in the database!!

# We cannot run tests in a Jupyter Notebook; uncommenting the following line in a regular script should work
#unittest.main()

The main issue we're facing is that this test runs on live data because the singleton is forcing us to, and if the data changes, our test would break.

We can fix this by feeding some dummy values into a dummy database, and then use that instead. In order to do that, we will have to modify our record finder so that we can configure it with the database we want to use. We will also create the dummy database with dummy fixed values.

In [5]:
class ConfigurableRecordFinder:
    def __init__(self, db):
        self.db = db

    def total_population(self, cities):
        result = 0
        for c in cities:
            result += self.db.population[c] # Note that we're not calling the Singleton database anymore, but our selected DB
        return result

class DummyDatabase:
    population = {
        'alpha': 1,
        'beta': 2,
        'gamma': 3
    }

    def get_population(self, name):
        return self.population[name]

With this, we can modify my tests so that they make use of our configurable record finder and our dummy database:

In [6]:
class SingletonTests(unittest.TestCase):
    def test_is_singleton(self):
        db = Database()
        db2 = Database()
        self.assertEqual(db, db2)

    ddb = DummyDatabase()

    def test_dependent_total_population(self):
        crf = ConfigurableRecordFinder(self.ddb)
        self.assertEqual(
            crf.total_population(['alpha', 'beta']),
            3
        )

# We cannot run tests in a Jupyter Notebook; uncommenting the following line in a regular script should work
#unittest.main()

The main takeaway from this lesson is that we must avoid direct dependencies on singleton classes, and that we should replace these direct dependencies by something else (in this case, by injecting a value into the initializer).